# **Load ExllamaV2**

In [ ]:
#install ExLLamaV2
!git clone https://github.com/turboderp/exllamav2
!pip install -e exllamav2
%cd /kaggle/working/exllamav2
!pip install .
!git lfs install

# **Load Llama3.1 8b at 8bpw**

In [ ]:
!git clone -b  8.0bpw https://huggingface.co/turboderp/Llama-3.1-8B-Instruct-exl2
    #ignore: Encountered 1 file(s) that may not have been copied correctly on Windows: output.safetensors

In [ ]:
import sys, os
sys.path.append('/content/exllamav2')

from exllamav2 import(
    ExLlamaV2,
    ExLlamaV2Config,
    ExLlamaV2Cache,
    ExLlamaV2Tokenizer,
)

from exllamav2.generator import (
    ExLlamaV2BaseGenerator,
    ExLlamaV2Sampler
)

In [ ]:
#instantiate the model
model_directory =  "Llama-3.1-8B-Instruct-exl2"
batch_size = 1

config = ExLlamaV2Config()
config.model_dir = model_directory
config.prepare()

config.max_batch_size = batch_size  #model needs to allocate temp buffers to fit the max batch size

model = ExLlamaV2(config)
print("Loading model: " + model_directory)

cache = ExLlamaV2Cache(model, lazy = True, batch_size = batch_size)  #cache needs to accommodate the batch size
model.load_autosplit(cache)


tokenizer = ExLlamaV2Tokenizer(config)

# Initialize generator
generator = ExLlamaV2BaseGenerator(model, cache, tokenizer)

In [ ]:
#model settings
settings = ExLlamaV2Sampler.Settings()
settings.temperature = 0.2
settings.top_k = 20
settings.top_p = 0.1
settings.token_repetition_penalty = 1.05 #base: 1.05

# **Prepare files**

In [ ]:
#prep files for input
prompts = []
folderPath = "/kaggle/input/bug-txts"

# Iterate over the files in the directory
for filename in os.listdir(folderPath):
    filePath = os.path.join(folderPath, filename)
    # Ensure that we are reading only files, not directories
    if os.path.isfile(filePath):
        with open(filePath, "r") as f:
            prompts.append(f.read())
        
#format the prompts
def format_prompt(sp,exUsrInpt, exResponse, p):
    return f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

    Cutting Knowledge Date: December 2023
    Today Date: 23 July 2024

    {sp}<|eot_id|><|start_header_id|>user<|end_header_id|>

    {exUsrInpt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
    
    {exResponse}<|eot_id|><|start_header_id|>user<|end_header_id|>

    {p}<|eot_id|><|start_header_id|>assistant<|end_header_id|>""" 


system_prompt = """You are a Neo4j knowledge graph expert. Your task is to generate Cypher statements based only on the user's input, which comes from a textbook about pests.
Understand the formatting of the user's input before generating Cypher, Use MERGE, use node properties, for example a pests scientific name is a property of the pest node. 
Limit cypher to the user's text, exluding the photos and avoiding repetition. Include only the Cypher statements in your response, create your cypher in the order of appearance in the text, for example if pesiticides are at the end their cypher should be generated last in the response, then end."""

exP = """88   Pests of Trees and Shrubs
Trunk injury and emergence holes caused by lilac/ash
clearwing borer. (47) Photo: John Davidson
Pupal skin of lilac/ash borer protruding from trunk of lilac. (49)
Photo: John DavidsonClearwing borers: lilac/ash, banded ash
Several species
Order Lepidoptera, Family Sesiidae;
clearwing borer moths
Native pests
Host plants: Alder, ash, birch, dogwood, lilac, hawthorn,
mountainash, maple, oak, pine, poplar, sycamore,
viburnum, willow, and fruit trees such as apricot, cherry,
peach, and plum
Description: Adult clearwing borers are wasplike, 12–19
mm long with long, narrow front wings and shorter,
wider hind wings. The hind wings, and in some speciesthe front wings, are mostly clear, with a wingspanbetween 26–40 mm. For some species, the two sexeshave different coloring, often yellow, orange or red on
black, or dark blue.
Life history:  Clearwing borers develop through four life
stages: egg, larva, pupa, and adult. Adults of various
species emerge from spring through summer and depositeggs on bark. The banded ash borer adults emerge inAugust. Adults do not directly damage plants and liveonly for a few days. Larvae mine the sapwood of trees
during the summer, pupating the following spring. There
is usually one generation a year.
Overwintering: Mature larvae in tunnels under the bark.
Damage symptoms:  Larval feeding of clearwing borers
can cause tree bark to become gnarled or rough, and
damage tissue that conducts food and water. With someclearwing borer species, feeding is tolerated by trees andcauses no serious harm. Feeding by other species can
weaken or kill branches, and may even kill the entire
tree.
Monitoring: Ash/lilac borer and lesser peachtree
borer adults begin to emerge when common lilac and
Vanhoutte spirea bloom in early to mid May. (Herms).Greater peachtree borer adults begin to emerge whenNorthern catalpa blooms in early to mid June. (Herms).
Banded ash clearwing borer adults begin to emerge in
late July (Columbus, Ohio) and in mid August (Wooster,Ohio) (Herms).
Monitoring and management for all clearwing borers is
similar, since their life cycle is about the same. However,the banded ash clearwing is the exception to the rule; theadults emerge in August rather than May to lay eggs. Look
for brown frass around bark cracks and bark wounds.
Look under loose bark near wounds or cracks for signs oflarval tunneling or empty pupal skins. Display phero-mone, or sex attractant, traps in May to capture adultmale moths. This trapping information can be used totime pesticide applications. Traps and pheromone lures
can be obtained from numerous sources. One source is
Great Lakes IPM, at www.greatlakesipm.com. Other
suppliers include Gempler’s at www.gemplers.comFrass and sawdust caused by lilac/ash clearwing borer larvae. (42)
Photo: James Solomon, USDA Forest Service, The BugwoodNetwork, University of Georgia.

IPM of Midwest Landscapes
 89   Pests of Trees and Shrubs
Clearwing borers: lilac/ash, banded ash, and dusky
Banded ash clearwing borer adult. (42)
Photo: John DavidsonCultural control:  Avoid mechanical damage to the bark
of trees, and minimize tree stress. Do not band trees as it
has been shown to increase borer attack.
Chemical control: Spray a long lasting, broad spectrum
insecticide on the trunk and limbs of infested trees 10
days after the first male is found in a pheromone trap.
Biological control: Several species of parasitic wasps
have been reared from larvae or pupae of the peachtree
borer. They include Bracon sanninoideae  and
Macrocentrus marginator  (Braconidae);  Cryptus
rufovinctus , Phaeogenes ater , and Venturia nigricoxalis
(Ichneumonidae); Syntomophyrum clisiocampae
(Eulophidae); and Telenomus quaintacei  (Scelonidae). The
entomopathogenic nematode, Steinernema carpocapsae ,
can control borer larvae if sprayed on frass-surroundedbark cracks in summer. The use of pheromones in amating disruption program maybe an effective control
for nurseries but not landscapes.
Plant mortality risk: High
Lilac/ash clearwing borer adult male. (48)
Photo: John Davidson
Dusky clearwing adult female on left; male on right. (23)
Photo: James Solomon, USDA Forest Service, The BugwoodNetwork, University of Georgia.Lilac/ash clearwing borer adult. (20)Photo: James Solomon, USDA Forest Service, The BugwoodNetwork, University of Georgia.Bark damage caused by banded ash clearwing borer. The borersentered a tree after an adult female laid an egg where a twigwas recently pruned. (42)
Photo: Cliff Sadof
Biorational pesticides:  Apply entomophagous nematodes
(Heterorhabditis bacteriophora, Steinernema carpocapsae )
with plenty of water, when temperatures are between 55º
and 85º F, and when larvae or pupae are present. Many
clearwing borers are caterpillars from late June through
the winter into April.
Conventional pesticides:  chlorpyrifos (nursery only),
permethrin
IPM of Midwest Landscapes
 90   Pests of Trees and Shrubs
Trunk and root collar damage caused by
cottonwood  borer. Larva shown indamaged stem. (066)Photo: James Solomon, USDA ForestService, The Bugwood Network,University of Georgia.
Cottonwood  borer adult female. (30)
Photo: James Solomon, USDA Forest Service, The BugwoodNetwork, University of Georgia.Bark cracks and frass caused by cottonwood
borer. (30)Photo: James Solomon, USDA Forest Service,The Bugwood Network, University of Georgia.Clearwing borers: cottonwood borers

IPM of Midwest Landscapes
 91   Pests of Trees and Shrubs
Dogwood  borer adult male. (306)
Photo: David LaughlinClearwing borers: dogwood borers
Dogwood  borer adult female. (43)
Photo: David Laughlin
Trunk injury caused by dogwood  borer. Wrapping dogwood
tightly in plastic barriers keeps wood moist and makes treesmore susceptible to injury from dogwood borer. (42)
Photo: Cliff SadofDogwood borer adult female. (09)
Photo: James Solomon, USDA Forest Service, The BugwoodNetwork, University of Georgia.Trunk damage caused by dogwood  borer larva.
(66)Photo: Charles Cornell

IPM of Midwest Landscapes
 92   Pests of Trees and Shrubs
Clearwing borers: dogwood borers, rhododendron borers
Dogwood  borer adult male. (10)
Photo: James Solomon, USDA Forest Service, The BugwoodNetwork, University of Georgia.
Dogwood borer larvae. (306)
Photo: David Laughlin
Dogwood borer larva and pupa compared to a standard pencil
eraser. (45)
Photo: John DavidsonPupal skin of dogwood borer protruding from trunk. (46)
Photo: Cliff Sadof
Trunk damage caused by rhododendron borer. (62)
Photo: John DavidsonRhododendron borer adult male. Note the three gold bands on
the abdomen. (63) Photo: John Davidson

IPM of Midwest Landscapes
 93   Pests of Trees and Shrubs
Clearwing borers: maple callus borers, oak borers
Maple callus borer adult male; note the spotted front wing. Males
are attracted to pheromone traps. (50)
Photo: John Davidson
Squash vine borer adult male. (167)
Photo: John Davidson
Maple callus borer adult male. (167)
Photo: John DavidsonOak clearwing borer adult male. Males are attracted to
pheromone traps. (51)
Photo: Cliff SadofOak clearwing borer adult. (52)
Photo: David Laughlin

IPM of Midwest Landscapes
 94   Pests of Trees and Shrubs
Peachtree borer larva. (56)
Photo: John DavidsonPeachtree borer adult female. (54)
Photo: David LaughlinClearwing borers: peachtree borers
Peachtree borer adult male. (55)Photo: John Davidson
Trunk and root collar damage caused by peachtree borer in a
sand cherry in a nursery. (42)
Photo: Cliff Sadof
Peachtree borer larvae. Soil removed to show
frass and exudate. (39)Photo: James Solomon, USDA Forest Service,The Bugwood Network, University of Georgia.Dieback caused by peachtree borer infestation at base of the
tree. (53)
Photo: John Davidson

IPM of Midwest Landscapes
 95   Pests of Trees and Shrubs
Peachtree borer larva killed by parasitic nematodes,
Steinernema carpocapsae. (59)
Photo: John DavidsonDead peachtree borer larva decomposing and releasing
parasitic nematode, Steinernema carpocapsae. (60)
Photo: John Davidson
Close-up of parasitic nematode, Steinernema carpocapsae,
entering peachtree larva through the spiracle (breathing pore).(58)
Photo: John DavidsonParasitic nematode,  Steinernema carpocapsae,  entering peach-
tree larva through the spiracle (breathing pore). (58)Photo: John DavidsonClearwing borers: peachtree borers
Spraying parasitic nematodes on cracks in tree bark to controlclearwing borer larvae. (57)
Photo: John Davidson
Close-up of nematodes, Steinernena carpocapsae, leaving lysed
body of dead pupa. (61)
Photo: Vera Krischik"""

exR = """MERGE (pest:Pest {name: "Lilac/Ash Clearwing Borer"})
MERGE (order:Order {name: "Lepidoptera"})
MERGE (family:Family {name: "Sesiidae"})
MERGE (pest_type:PestType {type: "Native pests"})
MERGE (risk:PlantMortalityRisk {risk: "High"})
MERGE (pest)-[:BELONGS_TO_ORDER]->(order)
MERGE (pest)-[:BELONGS_TO_FAMILY]->(family)
MERGE (pest)-[:HAS_TYPE]->(pest_type)
MERGE (pest)-[:HAS_RISK]->(risk)
MERGE (alder:Plant {name: "Alder"})
MERGE (ash:Plant {name: "Ash"})
MERGE (birch:Plant {name: "Birch"})
MERGE (dogwood:Plant {name: "Dogwood"})
MERGE (lilac:Plant {name: "Lilac"})
MERGE (hawthorn:Plant {name: "Hawthorn"})
MERGE (mountainash:Plant {name: "Mountainash"})
MERGE (maple:Plant {name: "Maple"})
MERGE (oak:Plant {name: "Oak"})
MERGE (pine:Plant {name: "Pine"})
MERGE (poplar:Plant {name: "Poplar"})
MERGE (sycamore:Plant {name: "Sycamore"})
MERGE (viburnum:Plant {name: "Viburnum"})
MERGE (willow:Plant {name: "Willow"})
MERGE (apricot:Plant {name: "Apricot"})
MERGE (cherry:Plant {name: "Cherry"})
MERGE (peach:Plant {name: "Peach"})
MERGE (plum:Plant {name: "Plum"})
MERGE (pest)-[:INFECTS]->(alder)
MERGE (pest)-[:INFECTS]->(ash)
MERGE (pest)-[:INFECTS]->(birch)
MERGE (pest)-[:INFECTS]->(dogwood)
MERGE (pest)-[:INFECTS]->(lilac)
MERGE (pest)-[:INFECTS]->(hawthorn)
MERGE (pest)-[:INFECTS]->(mountainash)
MERGE (pest)-[:INFECTS]->(maple)
MERGE (pest)-[:INFECTS]->(oak)
MERGE (pest)-[:INFECTS]->(pine)
MERGE (pest)-[:INFECTS]->(poplar)
MERGE (pest)-[:INFECTS]->(sycamore)
MERGE (pest)-[:INFECTS]->(viburnum)
MERGE (pest)-[:INFECTS]->(willow)
MERGE (pest)-[:INFECTS]->(apricot)
MERGE (pest)-[:INFECTS]->(cherry)
MERGE (pest)-[:INFECTS]->(peach)
MERGE (pest)-[:INFECTS]->(plum)
MERGE (damage1:DamageSymptom {description: "Trunk injury and emergence holes"})
MERGE (damage2:DamageSymptom {description: "Pupal skin protruding from trunk"})
MERGE (damage3:DamageSymptom {description: "Bark becomes gnarled or rough"})
MERGE (damage4:DamageSymptom {description: "Damage tissue that conducts food and water"})
MERGE (damage5:DamageSymptom {description: "Weaken or kill branches"})
MERGE (damage6:DamageSymptom {description: "May kill the entire tree"})
MERGE (pest)-[:CAUSES]->(damage1)
MERGE (pest)-[:CAUSES]->(damage2)
MERGE (pest)-[:CAUSES]->(damage3)
MERGE (pest)-[:CAUSES]->(damage4)
MERGE (pest)-[:CAUSES]->(damage5)
MERGE (pest)-[:CAUSES]->(damage6)
MERGE (monitoring:Monitoring {description: "Monitor adult emergence with pheromone traps in May, June, and August"})
MERGE (pest)-[:HAS_MONITORING]->(monitoring)
MERGE (cultural_control:CulturalControl {method: "Avoid mechanical damage to bark; minimize tree stress"})
MERGE (chemical_control:ChemicalControl {method: "Apply broad spectrum insecticide 10 days after first male is found in pheromone trap"})
MERGE (biological_control:BiologicalControl {method: "Use parasitic wasps, entomopathogenic nematodes, or pheromones for mating disruption"})
MERGE (biorational_pesticides:Pesticide {type: "Biorational", name: "Entomophagous nematodes"})
MERGE (conventional_pesticides:Pesticide {type: "Conventional", name: "Chlorpyrifos, Permethrin"})
MERGE (pest)-[:HAS_CONTROL]->(cultural_control)
MERGE (pest)-[:HAS_CONTROL]->(chemical_control)
MERGE (pest)-[:HAS_CONTROL]->(biological_control)
MERGE (pest)-[:HAS_CONTROL]->(biorational_pesticides)
MERGE (pest)-[:HAS_CONTROL]->(conventional_pesticides)
"""

#format and calculate max_tokens
f_prompts = [format_prompt(system_prompt, exP, exR, p) for p in prompts]
max_tokens = [len(tokenizer.encode(p, encode_special_tokens = True).size(1)) * 1.25 for p in prompts]

In [ ]:
#clear anything in the responses folder
!rm -rf /kaggle/working/responses

# **Start generating responses**

In [ ]:
#create response folder
output_folder = "/kaggle/working/responses"
os.makedirs(output_folder, exist_ok=True)

#generate cypher queries
for i, (filename, prompt) in enumerate(zip(os.listdir(folderPath), f_prompts)):
    
    #generate response
    full_response = generator.generate_simple(
        prompt, settings, max_tokens[i], completion_only = True
    )
    
    #get relevant part of the response
    response = full_response.replace("<<SYS>>", "")
    #removing everyhting after assistant
    sep = "assistant"
    keyword_pos = response.find(sep)
    if keyword_pos != -1:
        response = response[:keyword_pos].strip()
    
    #save response to file
    response_filename = os.path.join(output_folder, f"response_{filename}")
    with open(response_filename, "w") as response_file:
        response_file.write(response)
        
    #success
    print(f"Processed file: {filename}")

# **Write cypher to neo4j, create the knowledge graph**

In [ ]:
!pip install neo4j

In [ ]:
#connect to neo4j
from neo4j import GraphDatabase
from kaggle_secrets import UserSecretsClient

#login
user_secrets = UserSecretsClient()
uri = "neo4j+s://239252a9.databases.neo4j.io"
username = "neo4j"
password = user_secrets.get_secret("neo4j Password")
driver = GraphDatabase.driver(uri, auth=(username, password))

In [ ]:
#read Cypher query
def read_cypher_from_file(file_path):
    with open(file_path, 'r') as file:
        cypher_query = file.read().strip() #strip any extra whitespace
    return cypher_query

In [ ]:
#execute Cypher query
def execute_cypher_query(cypher_query):
    try:
        with driver.session() as session:
            result = session.run(cypher_query)
            return result.data()
    except Exception as e:
        return f"An error occurred: {e}"

In [ ]:
#process all files in a directory
def process_cypher_files(directory_path):
    
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):  #process only .txt files
            file_path = os.path.join(directory_path, filename)
            
            #read Cypher
            cypher_query = read_cypher_from_file(file_path)
            
            #execute Cypher
            result = execute_cypher_query(cypher_query)
            
            #check for error
            if isinstance(result, str) and result.startswith("An error occurred"):
                print(f"{result} in file: {filename}")
            else:
                print(f"Query in file {filename} executed successfully.")
                

In [ ]:
#send cypher to neo4j 
directory_path = "/kaggle/working/responses"
process_cypher_files(directory_path)

Add embeddings

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

def add_embeddings_to_nodes(driver):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    with driver.session() as session:
        # Fetch all nodes and their first property
        nodes = session.run("MATCH (n) RETURN id(n) AS id, keys(n)[0] AS first_property, n[keys(n)[0]] AS value")
        
        for node in nodes:
            ID = node["id"]
            value = node["value"]
            
            if value:  #ensure it exists
                embedding = model.encode(value).tolist()  #convert embedding to a list
                #add embedding to node
                session.run(
                    """
                    MATCH (n)
                    WHERE id(n) = $id
                    SET n.embedding = $embedding
                    """,
                    id=ID, embedding=embedding
                )

#generate embeddings and update nodes
add_embeddings_to_nodes(driver)
print("done")

In [ ]:
#close neo4j session
driver.close()